# LangGraph Tutorial: Conversational Context

**Objective:** Understand how MessagesState maintains conversation history across multiple turns.

**Key Concept:** State persists, enabling the agent to reference previous exchanges.

## Setup

Rebuild the graph from previous notebooks.

In [1]:
# Core imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv
from typing import Literal

load_dotenv("../.env")
print("✅ Environment loaded")

✅ Environment loaded


In [2]:
# Define tools
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """Convert currency from one type to another."""
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates or to_currency not in exchange_rates:
        return f"Error: Unsupported currency"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """Calculate the EMI (Equated Monthly Installment) for a loan."""
    if principal <= 0 or annual_interest_rate < 0 or tenure_months <= 0:
        return "Error: Invalid input parameters"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

✅ Tools defined


In [3]:
# Initialize LLM and build graph
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.3,
    max_tokens=1024,
    project=os.getenv("GOOGLE_PROJECT_ID"),
    location=os.getenv("GOOGLE_REGION")
)

tools = [currency_converter, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

def call_llm(state: MessagesState):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", END]:
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "agent")

app = workflow.compile()
print("✅ Graph compiled")

✅ Graph compiled


---

## Multi-Turn Conversation Example

**Conversation Flow:**
1. Turn 1: Currency conversion
2. Turn 2: Reference previous result for EMI calculation
3. Turn 3: Ask about previous calculations

### Turn 1: Initial Query

In [4]:
# Start conversation
result = {
    "messages": [
        HumanMessage(content="Convert 50000 USD to INR")
    ]
}

print("Turn 1")
print("=" * 70)
print("👤 User: Convert 50000 USD to INR")
print()

Turn 1
👤 User: Convert 50000 USD to INR



In [5]:
# Execute
result = app.invoke(result)

# Display response
print("🤖 Assistant:")
print(result["messages"][-1].content)
print("\n" + "=" * 70)

🤖 Assistant:
The equivalent of 50,000 USD is 4,156,000 INR.



### Turn 2: Reference Previous Result

In [6]:
# Add new user message to existing state
print("\nTurn 2")
print("=" * 70)
print("👤 User: Now calculate EMI for that INR amount at 9% for 2 years")
print()

result["messages"].append(
    HumanMessage(content="Now calculate EMI for that INR amount at 9% for 2 years")
)


Turn 2
👤 User: Now calculate EMI for that INR amount at 9% for 2 years



In [7]:
# Execute with full conversation history
result = app.invoke(result)

# Display response
print("🤖 Assistant:")
print(result["messages"][-1].content)
print("\n" + "=" * 70)

🤖 Assistant:
The EMI for a loan of 4,156,000 INR at 9% annual interest for 2 years is 189,865.79 INR per month.



### Turn 3: Follow-up Question

In [8]:
# Add another turn
print("\nTurn 3")
print("=" * 70)
print("👤 User: What was the original USD amount I asked about?")
print()

result["messages"].append(
    HumanMessage(content="What was the original USD amount I asked about?")
)


Turn 3
👤 User: What was the original USD amount I asked about?



In [9]:
# Execute
result = app.invoke(result)

# Display response
print("🤖 Assistant:")
print(result["messages"][-1].content)
print("\n" + "=" * 70)

🤖 Assistant:
You originally asked about 50,000 USD. 



## Analyze Conversation State

Examine the complete message history.

In [10]:
print("FULL CONVERSATION HISTORY:")
print("=" * 70)
print(f"Total messages: {len(result['messages'])}")
print("\nMessage breakdown:")

message_counts = {}
for msg in result["messages"]:
    msg_type = type(msg).__name__
    message_counts[msg_type] = message_counts.get(msg_type, 0) + 1

for msg_type, count in message_counts.items():
    print(f"  {msg_type}: {count}")

print("\n" + "=" * 70)

FULL CONVERSATION HISTORY:
Total messages: 10

Message breakdown:
  HumanMessage: 3
  AIMessage: 5
  ToolMessage: 2



## Visualize Message Flow

In [11]:
print("COMPLETE MESSAGE FLOW:")
print("=" * 70)

turn = 0
for i, msg in enumerate(result["messages"], 1):
    if isinstance(msg, HumanMessage):
        turn += 1
        print(f"\n--- TURN {turn} ---")
        print(f"[{i}] 👤 USER: {msg.content}")
        
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"[{i}] 🤖 AGENT: Calling {len(msg.tool_calls)} tool(s)")
        else:
            print(f"[{i}] 🤖 AGENT: {msg.content[:60]}...")
            
    elif isinstance(msg, ToolMessage):
        print(f"[{i}] 🔧 TOOL: Executed")

print("\n" + "=" * 70)

COMPLETE MESSAGE FLOW:

--- TURN 1 ---
[1] 👤 USER: Convert 50000 USD to INR
[2] 🤖 AGENT: Calling 1 tool(s)
[3] 🔧 TOOL: Executed
[4] 🤖 AGENT: The equivalent of 50,000 USD is 4,156,000 INR....

--- TURN 2 ---
[5] 👤 USER: Now calculate EMI for that INR amount at 9% for 2 years
[6] 🤖 AGENT: Calling 1 tool(s)
[7] 🔧 TOOL: Executed
[8] 🤖 AGENT: The EMI for a loan of 4,156,000 INR at 9% annual interest fo...

--- TURN 3 ---
[9] 👤 USER: What was the original USD amount I asked about?
[10] 🤖 AGENT: You originally asked about 50,000 USD. ...



## Test Context Awareness

Verify the agent can reference specific past exchanges.

In [12]:
# Turn 4: Specific reference
print("Turn 4")
print("=" * 70)
print("👤 User: What was the EMI you calculated?")
print()

result["messages"].append(
    HumanMessage(content="What was the EMI you calculated?")
)

result = app.invoke(result)

print("🤖 Assistant:")
print(result["messages"][-1].content)
print("\n" + "=" * 70)

Turn 4
👤 User: What was the EMI you calculated?

🤖 Assistant:
[{'type': 'text', 'text': 'The EMI calculated was 189,865.79 INR.', 'extras': {'signature': 'CpQOAY89a1+qypLZSnjPotELoIrVReUGVYNuKg9kbpSxvjZuf1nYAiHEYWMyenLXQLtgPVw5MugxbiarMTmKWQjRSZGnFGt2yevC6y/tRDItX1yB0fq785Rb/yUcmKORlnybC92c5S4cBfSdCnKIQmmaJS7GpJ9KViWGHL1eJooH9zW0QR8GI0lY5Kpw9p6sBkT/ECUderHNwqIGdjzmMFq3f9AQGuhz9ElGkM8hHevQDC3vh9hHnw1PgZxsv0/jaxBPnwcHB4dXn1PRUCQC9ThD16OrUu6WZ5BEIDrMiDkKot2w3N78jFpb/5kyTHAEZeXW24HTgq+90BE9I/Iv9t2/KY9RMDzv/aKEAmnB10j5XZDlssZ785ME5gCShW7MZEVRSHZkST1YtK9tUbnn2Y1lanrdsT4KcrJemnYtUKCiHHkaCrBS6Hdn8E9Ddt9r7J9M6d+iHYG+SWOBCOn/G7QR5x4AHdm+iuMIuNv1FQ4IgxPTOznwo7ZDi+S1CQTciRJ2n07rZFzU5a/YIiIuHNJurVEciN3AMIyusNQ13askhDVc+EWDTqGYbSWQs3WFS/1nCFv66xNbSJic6z7NvDrvYqHOh1OyX+zxe8TyuDc4w7KOKDZrZpAzDGGDERjyitt/0rmQdToKrN5JvA6qreYknxluLRyfExOYeHiZD1j8/mo4wkgaCExh1lVRzkBywKg45hF0uUmHdvdFiWc9+Su7uCNIbPpi2uI4jouqNfaohrSYPFgmUtgAm/Vx3ZEtxcsRBh7eFuGd9lABxNxCc6dw9pW2RXg1vMomCqIN1t1rgTExP246EFEQbo8KLISc6q/OwPQ/j5VbI

## How Context Works

**State Preservation:**
```python
# Each invocation:
result = app.invoke(result)
#                   ^^^^^^
#                   Pass SAME state object
```

**Message Accumulation:**
1. Turn 1: `[HumanMessage, AIMessage, ToolMessage, AIMessage]`
2. Turn 2: `[...previous messages..., HumanMessage, AIMessage, ...]`
3. Turn 3: `[...all previous..., HumanMessage, AIMessage]`

**LLM Sees Everything:**
- Every time `call_llm` runs, it gets `state["messages"]`
- This includes the entire conversation history
- Agent can reference any past exchange

## Another Multi-Turn Example

**Scenario:** Planning a car purchase

In [13]:
# Start new conversation
conversation = {"messages": []}

# Turn 1: Budget in local currency
print("SCENARIO: Car Purchase Planning")
print("=" * 70)
print("\nTurn 1")
print("👤 User: I have a budget of 2000000 INR for a car. What's that in USD?")

conversation["messages"].append(
    HumanMessage(content="I have a budget of 2000000 INR for a car. What's that in USD?")
)
conversation = app.invoke(conversation)

print("🤖 Assistant:", conversation["messages"][-1].content)
print()

SCENARIO: Car Purchase Planning

Turn 1
👤 User: I have a budget of 2000000 INR for a car. What's that in USD?
🤖 Assistant: 2,000,000 INR is approximately 24,061.60 USD. 



In [14]:
# Turn 2: EMI calculation
print("Turn 2")
print("👤 User: If I take a loan for my full budget at 8% for 5 years, what will be my monthly payment?")

conversation["messages"].append(
    HumanMessage(content="If I take a loan for my full budget at 8% for 5 years, what will be my monthly payment?")
)
conversation = app.invoke(conversation)

print("🤖 Assistant:", conversation["messages"][-1].content)
print()

Turn 2
👤 User: If I take a loan for my full budget at 8% for 5 years, what will be my monthly payment?
🤖 Assistant: Your monthly payment would be 40,552.79 INR.



In [15]:
# Turn 3: Follow-up
print("Turn 3")
print("👤 User: What if I reduce the tenure to 3 years?")

conversation["messages"].append(
    HumanMessage(content="What if I reduce the tenure to 3 years?")
)
conversation = app.invoke(conversation)

print("🤖 Assistant:", conversation["messages"][-1].content)
print("\n" + "=" * 70)

Turn 3
👤 User: What if I reduce the tenure to 3 years?
🤖 Assistant: If you reduce the tenure to 3 years, your monthly payment will be 62,672.73 INR.



## Key Observations

**Context Awareness:**
- ✅ Agent remembers "that INR amount" from Turn 1
- ✅ Agent understands "my full budget" refers to previous value
- ✅ Agent interprets "reduce the tenure" as modification of last calculation

**State Management:**
- Each `app.invoke(result)` call appends new messages
- Previous messages never disappear
- LLM sees complete history every time

**Conversational Flow:**
- No need to repeat information
- Natural follow-up questions work
- Agent maintains context across tool calls

## Starting Fresh Conversations

To start a new conversation, create a new state object.

In [16]:
# New conversation (no context from previous)
new_conversation = {
    "messages": [
        HumanMessage(content="Convert 500 EUR to GBP")
    ]
}

result_new = app.invoke(new_conversation)

print("NEW CONVERSATION (Fresh State):")
print("=" * 70)
print(f"Messages in conversation: {len(result_new['messages'])}")
print("\n🤖 Response:", result_new["messages"][-1].content)
print("\n✅ No context from previous car purchase conversation")

NEW CONVERSATION (Fresh State):
Messages in conversation: 4

🤖 Response: 500 EUR is equal to 429.35 GBP.

✅ No context from previous car purchase conversation


---

## ✅ Conversational Context Complete!

**Learned:**
- ✅ MessagesState preserves full conversation history
- ✅ Pass same state object for context continuity
- ✅ Agent understands references to past exchanges
- ✅ Natural multi-turn conversations work seamlessly
- ✅ New state object = fresh conversation

**State Pattern:**
```python
state = {"messages": [HumanMessage("Turn 1")]}
state = app.invoke(state)  # Adds AI response

state["messages"].append(HumanMessage("Turn 2"))
state = app.invoke(state)  # Sees full history
```

---

## 🎉 LangGraph Tutorial Series Complete!

**You've Learned:**
1. ✅ **Setup & Validation** - Environment and LLM initialization
2. ✅ **Tools Creation** - Building custom financial tools
3. ✅ **Graph Construction** - Assembling agentic workflows
4. ✅ **Single Tool Execution** - Basic agent → tool → response pattern
5. ✅ **Sequential & Parallel** - Multi-tool orchestration strategies
6. ✅ **Conversational Context** - Multi-turn conversations with state

**Core Principles:**
- 🔧 Tools extend LLM capabilities
- 📊 State enables conversation memory
- 🔀 Routing creates dynamic behavior
- 🔄 Cycles allow iterative reasoning

**Next Steps:**
- Build your own tools for specific domains
- Add more complex workflows (human-in-the-loop, sub-graphs)
- Integrate LangSmith for observability
- Deploy in production applications